# Supercharge Groq with Tavily MCP

This notebook is designed for Python developers who want to empower Groq inference with real-time web access, enabling Groq models to utilize up-to-date information as context. Live web information is critical for AI agents tasked with performing research, monitoring trends, or generating context-aware responses. 

We will achieve this through three simple steps:
1. Set up **Groq MCP client** for fast inference.
2. Set up **Tavily MCP server** for web search, web scraping, and web crawling.
3. Seemlessly **connect the client to the server** through the Responses API.

---

## Getting Started

Follow these steps to set up:
1. **Sign up** for Tavily at [app.tavily.com](https://app.tavily.com/home/) to get your free API key.
2. **Sign up** for Groq at [console.groq.com](https://console.groq.com/keys) to get your API key.
3. **Copy your API keys** from your Tavily and Groq account dashboards.
4. **Paste your API keys** into the cell below and run the cell.

In [1]:
# To export your API keys into a .env file, run the following cell (replace with your actual keys):
!echo "TAVILY_API_KEY=<your-tavily-api-key>" >> .env
!echo "GROQ_API_KEY=<your-groq-api-key>" >> .env

In [ ]:
import json
import os
import time
from datetime import datetime

# API Configuration
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# Check if API key is set
if not GROQ_API_KEY:
    print("Please set your Groq production API key")
else:
    print("Groq API key configured successfully!")
if not TAVILY_API_KEY:
    print("Please set your Tavily API key")
else:
    print("Tavily API key configured successfully!")

Select the foundation model to power inference. Let's try OpenAI's flagship open-weight MoE model, [gpt-oss-120b](https://console.groq.com/docs/model/openai/gpt-oss-120b).

In [2]:
# Model configuration
MODEL = "openai/gpt-oss-120b"

## Step 1: set up Groq's OpenAI-compatible MCP client

In [3]:
from openai import OpenAI

# set up Groq MCP client
client = OpenAI(base_url="https://api.groq.com/api/openai/v1", api_key=GROQ_API_KEY)

## Step 2: set up Tavily's remote MCP server

In [4]:
# set up Tavily MCP server
tools = [
    {
        "type": "mcp",
        "server_url": f"https://mcp.tavily.com/mcp/?tavilyApiKey={TAVILY_API_KEY}",
        "server_label": "tavily",
        "require_approval": "never",
    }
]

## Step 3: Connect Groq to Tavily through responses API

In [5]:
def connect_groq_to_tavily(client, tools, query):
    """
    Connect Groq client to Tavily server through responses API.

    This function demonstrates the speed and accuracy of combining:
    - Groq's fast LLM inference (500+ tokens/second)
    - Tavily MCP server for real-time web retrieval
    """

    start_time = time.time()

    # Call Groq with Tavily MCP integration using responses API
    response = client.responses.create(
        model=MODEL,
        input=query,
        tools=tools,
        stream=False,
        temperature=0.1,
        top_p=0.4,
    )

    total_time = time.time() - start_time

    # Get response content from responses API
    content = (
        response.output_text if hasattr(response, "output_text") else str(response)
    )

    # collect executed tools (MCP tool calls)
    executed_tools = []

    # Extract MCP calls from response if available
    if hasattr(response, "output") and response.output:
        for output_item in response.output:
            if hasattr(output_item, "type") and output_item.type == "mcp_call":
                executed_tools.append(
                    {
                        "type": "mcp",
                        "arguments": getattr(output_item, "arguments", "{}"),
                        "output": getattr(output_item, "output", ""),
                        "name": getattr(output_item, "name", ""),
                        "server_label": getattr(output_item, "server_label", ""),
                    }
                )
    return {
        "content": content,
        "total_time": total_time,
        "mcp_calls_performed": executed_tools,
        "timestamp": datetime.now().isoformat(),
    }

Let's implement a helper function to display MCP tool calls and their results. This will provide transparency into which tools were called, their arguments, and outputs.

In [6]:
def print_mcp_calls(mcp_calls):
    executed_tools = mcp_calls["mcp_calls_performed"]
    if executed_tools:
        print(f"\nTAVILY MCP CALLS: Found {len(executed_tools)} tool call(s):")
        print("-" * 50)
        for i, tool in enumerate(executed_tools, 1):
            print(f"\nTool Call #{i}")
            print(f"   Type: {tool['type']}")
            print(f"   Tool Name: {tool['name']}")
            print(f"   Server: {tool['server_label']}")
            try:
                if tool["arguments"]:
                    args = (
                        json.loads(tool["arguments"])
                        if isinstance(tool["arguments"], str)
                        else tool["arguments"]
                    )
                    print(f"   Arguments: {args}")

                # Print model results for transparency
                if tool["output"]:
                    output_data = (
                        json.loads(tool["output"])
                        if isinstance(tool["output"], str)
                        else tool["output"]
                    )
                    if isinstance(output_data, dict) and "models" in output_data:
                        print(f"   Models found: {len(output_data['models'])}")
                        for j, model in enumerate(
                            output_data["models"][:5], 1
                        ):  # Show top 5
                            model_name = model.get("id", model.get("name", "Unknown"))
                            print(f"      {j}. {model_name}")
                        if len(output_data["models"]) > 5:
                            print(
                                f"      ... and {len(output_data['models']) - 5} more models"
                            )
                    else:
                        print(f"   Output: {str(output_data)[:200]}...")
            except Exception as e:
                print(f"   Could not parse tool data: {e}")
    print(f"   Total time: {mcp_calls['total_time']:.2f} seconds")
    print(f"   Tavily MCP calls: {len(mcp_calls['mcp_calls_performed'])}")

# Examples

---

## Demo 1: AI startup funding announcements

In [7]:
from IPython.display import Markdown

ai_funding_news = connect_groq_to_tavily(
    client,
    tools,
    "What are some recent AI startup funding announcements?",
)

Let's display the agent's response in markdown format.

In [ ]:
Markdown(ai_funding_news["content"])

Let's examine the agent's intermediate steps, including how it calls different tools and configures tool arguments such as `search_depth`, `time_range`, `max_results`, and more.

In [ ]:
print_mcp_calls(ai_funding_news)

## Demo 2: Product Search


In [11]:
product_search = connect_groq_to_tavily(
    client,
    tools,
    "Find the iphone models currently available on apple.com",
)

Let's display the agent's response in markdown format.

In [ ]:
Markdown(product_search["content"])

Let's examine the agent's intermediate steps.

In [ ]:
print_mcp_calls(product_search)

## Demo 3: Try it Yourself

Now it's your turn! Replace the query with some real time information you want to discover.


In [ ]:
your_query = "Your Query Here"  # Change this!

custom_response = connect_groq_to_tavily(client, tools, your_query)

In [ ]:
Markdown(custom_response["content"])

In [ ]:
print_mcp_calls(custom_response)